In [ ]:
from copy import deepcopy
import math
import sys

import matplotlib.pylab as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

%matplotlib inline

sys.path.append('../crazyflie-audio/python/')

In [ ]:
import pyroomacoustics as pra
from signals import generate_signal, amplify_signal
from scipy.spatial.transform import Rotation

def rotate_mics(mics, orientation_deg=0):
    """
    :param mics: mic positions (n_mics, 2)
    :return mics_rotated: (n_mics, 2)
    """
    rot = Rotation.from_euler('z', orientation_deg, degrees=True)
    R = rot.as_matrix() # 3x3
    mics_aug = np.c_[mics, np.ones(mics.shape[0])].T # 3 x 4
    mics_rotated = R.dot(mics_aug)[:2, :] # 2 x 4
    return mics_rotated.T
    
def generate_signals(mics_rotated):
    room = pra.AnechoicRoom(fs=Fs, dim=2)
    
    source_signal = generate_signal(Fs, signal_type="random", duration=duration)
    source_signal = amplify_signal(source_signal, target_dB=0.0)
    
    room.add_source(source, signal=source_signal, name=f"source")

    beam_former = pra.Beamformer(mics_rotated, room.fs)
    room.add_microphone_array(beam_former)
    room.simulate()
    return deepcopy(room.mic_array.signals), mics_rotated.T


def plot_spectrum(spectrum, degree=0):
    plt.figure()
    plt.pcolormesh(angles, frequencies, np.log10(spectrum))
    plt.axvline(degree, color='red')
    plt.xlabel('angle [deg]')
    plt.ylabel('frequency [Hz]')

In [ ]:
from mic_array import get_square_array

Fs = 32000  # Hz, sampling frequency
select_bins = False #True

baseline = 0.108  # meters, square side of mic array
duration = 1  # seconds
gt_distance = 1  # meters, distance of source
add_noise_mics_positions = 0 #1e-2

mics_drone = get_square_array(baseline=baseline, delta=0) # 4 x 2
if add_noise_mics_positions > 0:
    mics_drone += np.random.normal(scale=add_noise_mics_positions, size=mics_drone.shape)

gt_angle_deg = 90
gt_angle_rad = gt_angle_deg * np.pi / 180.0
source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])

In [ ]:
#degrees = [0, 0]
#degrees = range(10)
#degrees = np.arange(0, 360, 45)
degrees = [0, 10, 20, 30]

# 4 x n_times
np.random.seed(1)
mics_list = [rotate_mics(mics_drone, orientation_deg=degree) for degree in degrees]
signals_mics_list = [generate_signals(mics.T) for mics in mics_list] 

n_buffer = 1024
time_index = 1000
buffers = [signals_mics[0][:, time_index:time_index + n_buffer] for signals_mics in signals_mics_list] 

In [ ]:
from itertools import cycle

plt.figure()
ls_cycle = cycle([":", "-", "-."])
for degree, buffer in zip(degrees, buffers):
    label = f"rot {degree}"
    ls = next(ls_cycle)
    for i in range(buffer.shape[0]):
        label_mic = f"{label}: mic{i}"
        plt.plot(range(buffer.shape[1]), buffer[i], label=label_mic, ls=ls, color=f"C{i}")
plt.xlim(0, 20)
plt.legend(bbox_to_anchor=[1, 1], loc='upper left')

plt.figure()
for i, mics in enumerate(mics_list):
    plt.scatter(*mics.T, label=f'mic{i}')
plt.scatter(*source.T, label='source')
plt.axis('equal')
plt.legend(bbox_to_anchor=[1, 1], loc='upper left')
pass

# combine spectra

In [ ]:
from audio_stack.beam_former import BeamFormer
from bin_selection import select_frequencies

beam_former = BeamFormer(mic_positions=mics_drone)
angles = beam_former.theta_scan * 180 / np.pi

frequencies = np.fft.rfftfreq(n=n_buffer, d=1 / Fs)
signals_fs = [np.fft.rfft(buffer).T for buffer in buffers] # n_frequences x 4

if select_bins:
    indices = select_frequencies(n_buffer, Fs, min_freq=200, max_freq=7000)
    frequencies = frequencies[indices]
    signals_fs = [signals_f[indices, :] for signals_f in signals_fs]
    
Rs = [beam_former.get_correlation(signals_f) for signals_f in signals_fs]

spectra = [beam_former.get_das_spectrum(R, frequencies) for R in Rs]

combination_method = "sum"
normalization_method = "zero_to_one"
beam_former.init_dynamic_estimate(combination_n=len(degrees), combination_method=combination_method, normalization_method=normalization_method)
for spectrum, degree in zip(spectra, degrees):
    beam_former.add_to_dynamic_estimates(spectrum, -degree)

combined_spectra = beam_former.get_dynamic_estimate()
pass

In [ ]:
for spectrum, degree in zip(spectra, degrees):
    plot_spectrum(spectrum, gt_angle_deg - degree)
    plt.title(f'rot {degree}')

plot_spectrum(combined_spectra, gt_angle_deg - degrees[-1])
plt.title(f'combined spectra')
pass

# combine raw signals

In [ ]:
from algos_basics import get_mic_delays

mics_array = np.concatenate([*mics_list])
beam_former = BeamFormer(mic_positions=mics_array)

ref_mics = mics_list[-1]
signals_fs_delayed = []

signals_f_tot_zero = np.empty((len(frequencies), 0))
signals_f_tot_delay = np.empty((len(frequencies), 0))

for i, (mics, signals_f) in enumerate(zip(mics_list[:-1], signals_fs[:-1])):
    mics_ref = np.r_[[mics[0]], [ref_mics[0]]]
    
    # TODO(FD) we assume knowledge of the source here. 
    # ideally, we would move this inside the beamforming algorithm!
    delay = get_mic_delays(mics_ref, azimuth=gt_angle_rad)[1]
    signals_f_delayed = np.multiply(signals_f, np.exp(-1j * 2 * np.pi * frequencies * delay)[:, None])
    signals_f_tot_delay = np.c_[signals_f_tot_delay, signals_f_delayed]
    
    signals_f_tot_zero = np.c_[signals_f_tot_zero, signals_f]

signals_f_tot_zero = np.c_[signals_f_tot_zero, signals_fs[-1]] # n_frequencies x 8
signals_f_tot_delay = np.c_[signals_f_tot_delay, signals_fs[-1]] # n_frequencies x 8

Rtot_zero = beam_former.get_correlation(signals_f_tot_zero)
combined_raw_zero = beam_former.get_das_spectrum(Rtot_zero, frequencies)

Rtot_delay = beam_former.get_correlation(signals_f_tot_delay)
combined_raw_delay = beam_former.get_das_spectrum(Rtot_delay, frequencies)

plot_spectrum(combined_raw_delay)
plt.title('combined raw with correct delay')

plot_spectrum(combined_raw_zero)
plt.title('combined raw with zero delay')
pass

In [ ]:
from audio_stack.spectrum_estimator import combine_rows, normalize_rows

plt.figure()
combine = 'sum' 
normalize = 'sum_to_one'
keepdims = True
plt.plot(angles, normalize_rows(combine_rows(combined_spectra, method=combine, keepdims=keepdims),    
                                method=normalize).flatten(), label='combined spectra')
plt.axvline(gt_angle_deg - degrees[-1], color='C0', ls=':')
plt.plot(angles, normalize_rows(combine_rows(combined_raw_delay, method=combine, keepdims=keepdims),
                                method=normalize).flatten(), label='combined raw, correct delay')
plt.axvline(gt_angle_deg, color='C1', ls=':')
plt.plot(angles, normalize_rows(combine_rows(combined_raw_zero, method=combine, keepdims=keepdims),
                                method=normalize).flatten(), label='combined raw, zero delay')
plt.axvline(gt_angle_deg, color='C2', ls=':')
plt.yscale('log')
plt.ylim(1e-3, None)
plt.title('comparison spectra vs. raw combination')
plt.xlabel('angle [deg]')
plt.ylabel('probability [-]')
plt.legend()